In [246]:
from web3 import Web3
from web3.eth import Eth
from eth_account import Account
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256, SHA512
import struct
import base64
import hmac
import hashlib


In [202]:
BS = 16
def pad(data):
    padding = BS - len(data) % BS
    return data + padding * struct.pack("B", padding)

def unpad(data):
    return data[0:-data[-1]]

def AES256CbcDecrypt(hex_data, key=b'0'*32, iv=b'0'*16):
    data = bytearray.fromhex(hex_data)
    aes = AES.new(key, AES.MODE_CBC, iv)
    return unpad(aes.decrypt(data))

def AES256CbcEncrypt(bin_data, key=b'0'*32, iv=b'0'*16):
    aes = AES.new(key, AES.MODE_CBC, iv)
    return aes.encrypt(pad(bin_data))

In [203]:
#Test pading /unpading
start = bytearray(b'test')
padded = pad(start)
print(len(padded))
unpaded = unpad(padded)
print (unpaded)
print (unpaded==start)

16
bytearray(b'test')
True


In [204]:
#Test encrypt
hex_key = '1ac27dc3580fa322d795d54fe4b4b211980a299f924f1df267195c4f8253b700'
key = bytearray.fromhex(hex_key)
text ='test'
plaintext = text.encode(encoding='utf_8') 
hex_iv='95eb6bfc4ddcd00ba1b2d63b7c26bff4'
iv= bytearray.fromhex(hex_iv)

cry = AES256CbcEncrypt(plaintext,key,iv).hex()
print(cry)
print(cry=='e4bec3c272130e40fa34f068819d8e83')

#test decrypt
dec = AES256CbcDecrypt(cry,key,iv)
print(dec.decode("utf-8"))
print(dec.decode("utf-8")==text)

e4bec3c272130e40fa34f068819d8e83
True
test
True


In [93]:
def encode_key(pub_key):
    return pub_key.pointQ.x.to_bytes(32)+ pub_key.pointQ.y.to_bytes(32)

In [94]:
test_encoded = (74112620180903413835928205978618879887273296202846370176989995621597768248548).to_bytes(32,byteorder='big')+(70352546531620714376440768798610125065500941604960487601320179994127409968403).to_bytes(32,byteorder='big')
[ test_encoded[i] for i in range(len(test_encoded)) ] == [163, 218, 65, 170, 47, 136, 127, 159, 98, 77, 71, 242, 112, 244, 229, 36, 177, 2, 147, 111, 8, 233, 154, 255, 238, 244, 221, 88, 15, 2, 236, 228, 155, 138, 33, 87, 29, 185, 26, 29, 154, 176, 239, 200, 165, 137, 137, 181, 96, 137, 184, 159, 197, 133, 182, 251, 181, 179, 184, 113, 179, 141, 117, 19]

True

In [162]:
def derive(private_key, pub_ecc_point):
    return (pub_ecc_point*private_key.d).x

In [108]:
# not used
def concatenate0x04(data):
    return (0x04).to_bytes(1,byteorder='big')+data

In [113]:
hex_key = '1ac27dc3580fa322d795d54fe4b4b211980a299f924f1df267195c4f8253b700'
key = bytearray.fromhex(hex_key)
conc = concatenate0x04(key)
print(conc[0]==4 and conc[1]==key[0])

True


In [279]:
mac = hmac.new(b'key',b'test', hashlib.sha256).digest()
print([ mac[i] for i in range(len(mac)) ] == [2, 175, 181, 99, 4, 144, 44, 101, 111, 203, 115, 124, 221, 3, 222, 98, 5, 187, 109, 64, 29, 162, 129, 46, 253, 155, 45, 54, 160, 138, 241, 89])

True


In [280]:
mackey = bytearray([79, 239, 151, 70, 153, 199, 149, 191, 63, 138, 31, 222, 246, 48, 101, 30, 50, 242, 204, 220, 2, 110, 67, 183, 43, 138, 60, 2, 197, 100, 91, 198])
dataToMac=bytearray([149, 235, 107, 252, 77, 220, 208, 11, 161, 178, 214, 59, 124, 38, 191, 244, 4, 75, 110, 138, 127, 216, 95, 226, 105, 190, 209, 196, 201, 218, 135, 24, 64, 123, 111, 244, 29, 172, 190, 228, 139, 187, 179, 195, 2, 222, 119, 128, 51, 141, 128, 187, 166, 9, 238, 207, 172, 236, 18, 138, 135, 185, 9, 245, 30, 102, 165, 121, 224, 29, 40, 205, 213, 202, 79, 101, 78, 96, 28, 198, 185, 228, 190, 195, 194, 114, 19, 14, 64, 250, 52, 240, 104, 129, 157, 142, 131])
mac = hmac.new(mackey,dataToMac, hashlib.sha256).digest()
[ mac[i] for i in range(len(mac)) ] ==  [14, 120, 220, 2, 214, 66, 118, 74, 3, 156, 174, 142, 155, 38, 96, 183, 178, 132, 74, 102, 3, 195, 47, 32, 45, 59, 109, 239, 77, 155, 27, 130]

True

---------------------------------

In [281]:
def EncryptMessage(publicKey_hex, plainText_string):
    ephemPrivKey = ECC.generate(curve='secp256r1')
    ephemPubKey = ephemPrivKey.public_key()
    ephemPubKeyEncoded = encode_key(ephemPubKey)
    
    publicKey = bytearray.fromhex(publicKey_hex)
    pub = ECC.EccPoint(int.from_bytes(publicKey[:int(len(publicKey)/2)],byteorder='big'), int.from_bytes(publicKey[int(len(publicKey)/2):],byteorder='big'))
    px = derive(ephemPrivKey,pub) 
    hash_px = SHA512.new(data=px.to_bytes()).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    iv = Random.get_random_bytes(16)
    
    plaintext = plainText_string.encode(encoding='utf_8') 
    ciphertext = AES256CbcEncrypt(plaintext,encryptionKey,iv)
    dataToMac = iv + ephemPubKeyEncoded + ciphertext
    mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    serializedCiphertext = iv + ephemPubKeyEncoded + mac + ciphertext
    return serializedCiphertext.hex()
    

In [282]:
pub_key = '1a9894ac0dbbc75ee2cb41982a0de17b238559e13960fde765d8c68821a10522c5a2961cec479b29417eef79df3a5b31b4d5dfc62daa7bca8bb642a7b62f22b0'
message = 'test'
EncryptMessage(pub_key, message)

'a7e9ae5ee9af9c860c1e4465c5f1e3445cef853e6c1c4d7d6fc691f50a350e467ca64450741ee19867bc036b1c01e1bdcbbab0d4af2a7d56f49b1ae286b018286fff36f884f0a8efc6cd1027ede9e6ddd0dba4775a148a7849e8bb0cde1386462f9530bfc1bfec338b2d3e31351fa0c4daab853afc8692465edbdca210299e29'

------------------------------

In [78]:
ephemPrivKey = ECC.generate(curve='secp256r1')

In [80]:
ephemPubKey = ephemPrivKey.public_key()

In [95]:
ephemPubKeyEncoded = encode_key(ephemPubKey)

-----------------------------------------------

In [136]:
publicKey = bytearray.fromhex('1a9894ac0dbbc75ee2cb41982a0de17b238559e13960fde765d8c68821a10522c5a2961cec479b29417eef79df3a5b31b4d5dfc62daa7bca8bb642a7b62f22b0')

In [155]:
pub = ECC.EccPoint(int.from_bytes(publicKey[:int(len(publicKey)/2)],byteorder='big'), int.from_bytes(publicKey[int(len(publicKey)/2):],byteorder='big'))

In [163]:
px = derive(ephemPrivKey,pub) 

In [194]:
hash_px = SHA512.new(data=px.to_bytes()).digest()

In [199]:
encryptionKey = hash_px[:32]
macKey = hash_px[32:]


--------------------------------------------

In [195]:
iv = Random.get_random_bytes(16)

In [200]:
plaintext=b'This is my text'

In [205]:
ciphertext = AES256CbcEncrypt(plaintext,encryptionKey,iv)

In [207]:
dataToMac = iv + ephemPubKeyEncoded + ciphertext

---------------------------------

In [214]:
mac = hmac.new(macKey,dataToMac,'sha256').digest()

In [215]:
serializedCiphertext = iv + ephemPubKeyEncoded + mac + ciphertext

In [217]:
serializedCiphertext.hex()

'95eb6bfc4ddcd00ba1b2d63b7c26bff40b76ec0bd7a18a2c3d4ee88c84f3bea94541ece77025dafe825f2fa599220f96ea9c5bb9652b7eda838673575c98fcbb209867f69784f50827bdaf55074f37ccd639583522083cc784a2b41f45ab9798f297fe864076265a2fc6872989de2b3b0ba257fa1c2ae7ff357316bf187edbe9'